In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

import torch

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

from tqdm.notebook import tqdm
import torch.optim as optim

from utils import train, plot_images_full, generate_matrix, batch_dirichlet, get_similarities, plot_similarities, get_differences, plot_differences, get_images, plot_images

from models.vit_old import ViT
from models.diffusion import SimpleTransformer

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
PATH = "model_one.pth"
batch_size = 64
size = (3,32,32)
patch_size = 2
depth = 64
device = 'cuda:0'
data_path = './data/'
num_epochs = 50
learning_rate = 1e-4

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root=data_path, train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=data_path, train=False,
                                       download=True, transform=transform)
val_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## 1

In [4]:
model = SimpleTransformer(size, patch_size, depth, 
                          dim=128, 
                          heads=1,
                          num_classes=10, 
                          sign=0, 
                          tau=1, 
                          embed=True,
                          softw=False,
                          weight_sharing=False, 
                          method='A',
                          norm=True)

In [5]:
# model = ViT(
#     image_size = size[1],
#     patch_size = patch_size,
#     num_classes = 10,
#     dim = 256,
#     depth = depth,
#     heads = 8,
#     mlp_dim = 256,
#     dropout = 0.1,
#     emb_dropout = 0.1,
#     pool = 'mean'
# )

In [6]:
model = model.to(device)

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.05)

train_accs_list = []
train_loss_list = []
val_accs_list = []
val_loss_list = []

for epoch in tqdm(range(1, num_epochs + 1), desc="Epoch"):  
    train_loss, train_acc = train(model,train_loader,optimizer,criterion,epoch,device)  

    # Model validation
    model.eval()
    val_loss = val_acc = 0.0
    for (img, labels) in val_loader:
        img, labels = img.to(device), labels.to(device)
        with torch.no_grad():
            predictions = model(img)
            loss = criterion(predictions, labels)
            correct = torch.argmax(predictions.data, 1) == labels
        val_loss += loss
        val_acc += correct.sum()
    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader.dataset)
    
    train_accs_list.append(train_acc)
    val_accs_list.append(val_acc)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    
    if (epoch%2==0) or epoch==num_epochs or epoch==1:
        print(f"Epoch {epoch}")
        print(f" Train acc {train_acc:.3f}, Train loss {train_loss:.6f}")
        print(f" Valid acc {val_acc:.3f}, Val loss {val_loss:.6f}")

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1
 Train acc 0.262, Train loss 0.030024
 Valid acc 0.286, Val loss 0.029833


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [4]:
model.dim

NameError: name 'model' is not defined

In [10]:
torch.save(model.state_dict(),PATH)

In [11]:
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [8]:
m='A'
s=-1
ws=True
f"model_method_{m}_sign_{s}_ws_{ws}.pth"

'model_method_A_sign_-1_ws_True.pth'

In [1]:
round(3.44444)

3

In [29]:
A,B = torch.rand((64,64)),torch.rand((64,64))

In [30]:
A,B = torch.nn.functional.softmax(A,-1), B/torch.linalg.norm(B)

In [31]:
torch.linalg.norm(A)

tensor(1.0398)

In [32]:
torch.linalg.norm(B)

tensor(1.)

In [33]:
torch.linalg.norm(A+B)

tensor(1.9552)